In [1]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
# from nordpool import elspot, elbas
# import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy import stats
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy.stats import pearsonr,spearmanr,chi2, chi2_contingency
# from calendar import day_abbr, month_abbr, mdays
# import holidays

# from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

In [2]:
price=pd.read_csv('./data/electricity price/price.csv')
source=pd.read_csv('./data/source/sourceshare.csv')
netimport=pd.read_csv('./data/net import/netimport.csv')
carbon=pd.read_csv('./data/carbon/Carbon.csv')
weather=pd.read_csv('./data/weather data/weather.csv')
GDP=pd.read_csv('./data/gdp per capita/GDP.csv')
EVCarsaleshare=pd.read_csv('./data/EV/EVCarsaleshare.csv')

In [3]:
price['SE1']==price['SE2']

0       True
1       True
2       True
3       True
4       True
        ... 
1237    True
1238    True
1239    True
1240    True
1241    True
Length: 1242, dtype: bool

0       True
1       True
2       True
3       True
4       True
        ... 
1237    True
1238    True
1239    True
1240    True
1241    True
Length: 1242, dtype: bool

# Interpolation method for yearly/quarterly data
Many time series analyses assume observations over uniform time intervals.

In [4]:
Source=source[source['Year']>=2018]

In [5]:
Source.loc[:,'Year']=[ str(year)+'-06-15' for year in Source.loc[:,'Year']]


D:\ProgramData\Anaconda3\envs\daps\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
D:\ProgramData\Anaconda3\envs\daps\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [6]:
Source

,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables including bioenergy (% electricity)
33,2018-06-15,0.208,0.233,38.082,0.251,10.174,1.800,41.963,7.291
34,2019-06-15,0.125,0.172,38.814,0.404,11.786,1.692,39.265,7.743
35,2020-06-15,0.000,0.061,44.197,0.641,16.808,1.429,30.038,6.826
36,2021-06-15,0.000,0.058,42.965,0.614,15.964,1.368,31.239,7.792


,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables including bioenergy (% electricity)
33,2018-06-15,0.208,0.233,38.082,0.251,10.174,1.800,41.963,7.291
34,2019-06-15,0.125,0.172,38.814,0.404,11.786,1.692,39.265,7.743
35,2020-06-15,0.000,0.061,44.197,0.641,16.808,1.429,30.038,6.826
36,2021-06-15,0.000,0.058,42.965,0.614,15.964,1.368,31.239,7.792


In [7]:
null= np.full( 9, np.nan)

In [8]:
S=pd.DataFrame(np.insert(Source.values,4, null, axis=0))
S.columns = Source.columns
S.iloc[4,0]='2022-04-30'

In [9]:
S

,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables including bioenergy (% electricity)
0,2018-06-15,0.208,0.233,38.082,0.251,10.174,1.8,41.963,7.291
1,2019-06-15,0.125,0.172,38.814,0.404,11.786,1.692,39.265,7.743
2,2020-06-15,0.0,0.061,44.197,0.641,16.808,1.429,30.038,6.826
3,2021-06-15,0.0,0.058,42.965,0.614,15.964,1.368,31.239,7.792
4,2022-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables including bioenergy (% electricity)
0,2018-06-15,0.208,0.233,38.082,0.251,10.174,1.8,41.963,7.291
1,2019-06-15,0.125,0.172,38.814,0.404,11.786,1.692,39.265,7.743
2,2020-06-15,0.0,0.061,44.197,0.641,16.808,1.429,30.038,6.826
3,2021-06-15,0.0,0.058,42.965,0.614,15.964,1.368,31.239,7.792
4,2022-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
S.index=pd.to_datetime(S['Year'].astype('str'))
del S['Year']

In [11]:
import numpy as np
from scipy.interpolate import griddata

def func(x, y):
    return x*(1-x)*np.cos(4*np.pi*x) * np.sin(4*np.pi*y**2)**2

grid_x, grid_y = np.mgrid[0:1:100j, 0:1:200j]

points = np.random.rand(100, 2)
values = func(points[:,0], points[:,1])

grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')

fill_value = 123  # Whatever you like
grid_z0[np.isnan(grid_z1)] = fill_value

In [12]:
grid_z0

array([[123., 123., 123., ..., 123., 123., 123.],
       [123., 123., 123., ..., 123., 123., 123.],
       [123., 123., 123., ..., 123., 123., 123.],
       ...,
       [123., 123., 123., ..., 123., 123., 123.],
       [123., 123., 123., ..., 123., 123., 123.],
       [123., 123., 123., ..., 123., 123., 123.]])

array([[123., 123., 123., ..., 123., 123., 123.],
       [123., 123., 123., ..., 123., 123., 123.],
       [123., 123., 123., ..., 123., 123., 123.],
       ...,
       [123., 123., 123., ..., 123., 123., 123.],
       [123., 123., 123., ..., 123., 123., 123.],
       [123., 123., 123., ..., 123., 123., 123.]])

In [13]:
# S_RE = S.resample('1D')

# S_RE = S_RE.interpolate(method='time')

In [14]:
# print(S_RE)

In [15]:
P=price[['Date','SE1']]

In [16]:
P=P[P['Date']<='2022-04-30']

In [17]:
W=weather[weather['Date']<='2022-04-30']

In [18]:
W

,Date,temp,precip,windspeed
0,2019-01-01,3.9,0.46,34.5
1,2019-01-02,-1.1,0.29,46.6
2,2019-01-03,-2.7,0.69,12.9
3,2019-01-04,2.7,1.64,18.5
4,2019-01-05,0.8,0.17,15.6
...,...,...,...,...
1211,2022-04-26,7.3,0.00,19.0
1212,2022-04-27,4.9,0.00,20.2
1213,2022-04-28,7.0,0.00,26.3
1214,2022-04-29,6.0,0.00,16.4


,Date,temp,precip,windspeed
0,2019-01-01,3.9,0.46,34.5
1,2019-01-02,-1.1,0.29,46.6
2,2019-01-03,-2.7,0.69,12.9
3,2019-01-04,2.7,1.64,18.5
4,2019-01-05,0.8,0.17,15.6
...,...,...,...,...
1211,2022-04-26,7.3,0.00,19.0
1212,2022-04-27,4.9,0.00,20.2
1213,2022-04-28,7.0,0.00,26.3
1214,2022-04-29,6.0,0.00,16.4


In [28]:
table=pd.merge(P,W).rename(columns = {'SE1':'Price'})

In [20]:
Price=table['Price']
temp=table['temp']
precip=table['precip']
windspeed=table['windspeed']

temp

In [21]:
np.cov(Price, temp)

array([[456.31274357, -27.75391367],
       [-27.75391367,  54.55891445]])

array([[456.31274357, -27.75391367],
       [-27.75391367,  54.55891445]])

In [22]:
corr1, _ = pearsonr(Price, temp)
print("pearsonr correlation is", corr1)

pearsonr correlation is -0.17589766482153862
pearsonr correlation is -0.17589766482153862


In [23]:
corr2, _ = spearmanr(Price, temp)
print("spearmanr correlation", corr2)

spearmanr correlation -0.12984467168972372
spearmanr correlation -0.12984467168972372


In [29]:
size(table

,Date,Price,temp,precip,windspeed
0,2019-01-01,22.58,3.9,0.46,34.5
1,2019-01-02,42.65,-1.1,0.29,46.6
2,2019-01-03,51.90,-2.7,0.69,12.9
3,2019-01-04,50.85,2.7,1.64,18.5
4,2019-01-05,50.72,0.8,0.17,15.6
...,...,...,...,...,...
1211,2022-04-26,23.94,7.3,0.00,19.0
1212,2022-04-27,43.43,4.9,0.00,20.2
1213,2022-04-28,19.30,7.0,0.00,26.3
1214,2022-04-29,31.43,6.0,0.00,16.4


,Date,Price,temp,precip,windspeed
0,2019-01-01,22.58,3.9,0.46,34.5
1,2019-01-02,42.65,-1.1,0.29,46.6
2,2019-01-03,51.90,-2.7,0.69,12.9
3,2019-01-04,50.85,2.7,1.64,18.5
4,2019-01-05,50.72,0.8,0.17,15.6
...,...,...,...,...,...
1211,2022-04-26,23.94,7.3,0.00,19.0
1212,2022-04-27,43.43,4.9,0.00,20.2
1213,2022-04-28,19.30,7.0,0.00,26.3
1214,2022-04-29,31.43,6.0,0.00,16.4


In [25]:
sns.scatterplot(table)

D:\ProgramData\Anaconda3\envs\daps\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
D:\ProgramData\Anaconda3\envs\daps\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


ValueError: If using all scalar values, you must pass an index

ValueError: If using all scalar values, you must pass an index

precip

In [ ]:
np.cov(Price,precip)

In [ ]:
corr1, _ = pearsonr(Price,precip)
print("pearsonr correlation is", corr1)

In [ ]:
corr2, _ = spearmanr(Price,precip)
print("spearmanr correlation", corr2)

wind

In [ ]:
np.cov(Price,windspeed)

In [ ]:
corr1, _ = pearsonr(Price,windspeed)
print("pearsonr correlation is", corr1)

In [ ]:
corr2, _ = spearmanr(Price,windspeed)
print("spearmanr correlation", corr2)

In [ ]:
sns.set_style('whitegrid') 
fig=plt.figure(figsize=(12,6))
sns.lineplot(data=table,palette='tab10',linewidth=1.5,)
plt.ylim(10,110)
# plt.ylabel('Auction Price €/tCO2')
# sns.set_context('paper', font_scale=1.5, rc={'lines.linewidth':1.5})
# sns.despine(offset=(10,10),trim=False)